In [ ]:
import pandas as pd
import numpy as np
from datetime import *
from plotnine import *
import statsmodels.api as sm
import statsmodels.formula.api as smf
import requests

In [ ]:
df = pd.concat([pd.read_csv("data/lichess_swiss_rating_histories_1.csv",parse_dates=['date']),
                pd.read_csv("data/lichess_swiss_rating_histories_2.csv",parse_dates=['date'])])
print(df.shape)
df.head()

In [ ]:
df['user_id'].nunique()

In [ ]:
df['date'].max()

In [ ]:
# The latest date we have data on
max_outcome_date = df['date'].max()
# The latest date that can be used for training to ensure we'll always have 2 years in advance of outcomes data
max_training_date = max_outcome_date - timedelta(days=365*2)
max_outcome_date,max_training_date

In [ ]:
# The earliest date we have data on
minn_training_date = df['date'].min()
minn_training_date

In [ ]:
# The latest ratings that can be used for training
df_training = df.query('date<=@max_training_date')
df_outcomes = df.query('date>@max_training_date')
latest_training_ratings = df_training.sort_values("date",ascending=False).drop_duplicates(['user_id','time_control'])
latest_training_ratings.sample(5)

# Feature Engineering

In [ ]:
# Ratings from X days before the max training date
max_training_date_minus_30 = max_training_date-timedelta(days=30)
max_training_date_minus_90 = max_training_date-timedelta(days=90)
max_training_date_minus_180 = max_training_date-timedelta(days=180)
hist_ratings_30 = df.query('date<=@max_training_date_minus_30').sort_values("date",ascending=False).drop_duplicates(['user_id','time_control'])
hist_ratings_90 = df.query('date<=@max_training_date_minus_90').sort_values("date",ascending=False).drop_duplicates(['user_id','time_control'])
hist_ratings_180 = df.query('date<=@max_training_date_minus_180').sort_values("date",ascending=False).drop_duplicates(['user_id','time_control'])
hist_ratings_180.head()

In [ ]:
# Peak ratings
hist_ratings_peak = df_training.sort_values("rating",ascending=False).drop_duplicates(['user_id','time_control'])
outcome_ratings_peak = df_outcomes.sort_values("rating",ascending=False).drop_duplicates(['user_id','time_control'])
hist_ratings_peak.head()

In [ ]:
# Add features to base table
df_base = latest_training_ratings.merge(hist_ratings_30[['user_id','time_control','rating']],
                how='left',on=['user_id','time_control'],suffixes=['_latest','_30']).merge(
            hist_ratings_90[['user_id','time_control','rating']],
                how='left',on=['user_id','time_control']).merge(
            hist_ratings_180[['user_id','time_control','rating']],
                how='left',on=['user_id','time_control'],suffixes=['_90','_180']).merge(
            hist_ratings_peak[['user_id','time_control','rating']].rename(columns={'rating':'rating_peak'}),
                how='left',on=['user_id','time_control'])
df_base['rating_30_diff'] = df_base['rating_latest']-df_base['rating_30']
df_base['rating_90_diff'] = (df_base['rating_latest']-df_base['rating_90']).combine_first(df_base['rating_30_diff'])
df_base['rating_180_diff'] = (df_base['rating_latest']-df_base['rating_180']).combine_first(df_base['rating_90_diff'])
df_base['rating_peak_diff'] = df_base['rating_latest']-df_base['rating_peak']
df_base['time_control_copy'] = df_base['time_control']
df_base['rating_latest_rounded'] = df_base['rating_latest'].round(-2)
df_base['rating_latest_squared'] = df_base['rating_latest']**2
df_base = pd.get_dummies(df_base,columns=['time_control_copy'],prefix_sep="")
df_base.columns = [x.replace("time_control_copy","").lower() for x in df_base.columns]
print(df_base.shape)
df_base.sample(10)

In [ ]:
# Filter to people who have played rated games in the time control before 30 days ago...
# ... and have played at least one rated game in the time control within the last 30 days
df_base = df_base[(df_base['rating_30'].notna())&(df_base['date']>=max_training_date_minus_30)]
df_base.shape

In [ ]:
# What is the distribution of rating gains over the two year period?
## Use this to come up with reasonable target rating ranges where I'll have a decent sample size to work with when estimating how long it'll take
df_max_rating_gains = df_base.merge(outcome_ratings_peak,on=['user_id','time_control'],how='inner')
df_max_rating_gains['max_gain'] = df_max_rating_gains['rating']-df_max_rating_gains['rating_latest']
df_max_rating_gains['rating_bucket'] = df_max_rating_gains['rating_latest'].apply(lambda x: 1 if x < 1550 else (2 if x < 1900 else 3))
df_max_rating_gains.groupby("rating_bucket")['max_gain'].describe(percentiles=[.25,.5,.75,.9,.95,.99])

In [ ]:
# Generate target ratings
df_targets = pd.concat([df_base for x in range(5)])
def get_target_rating_gain(x):
    die = np.random.randint(1,4)
    if die == 1:
        return np.random.randint(1,100)
    elif die == 2:
        return np.random.randint(1,300)
    elif die == 3:
        if x < 1550:
            return np.random.randint(1,650)
        elif x < 1900:
            return np.random.randint(1,450)
        else:
            return np.random.randint(1,350)
    else:
        print(1/0)

df_targets['target_rating_gain'] = df_targets['rating_latest'].apply(get_target_rating_gain)
df_targets['target_rating'] = df_targets['rating_latest'] + df_targets['target_rating_gain']
df_targets['target_rating_gain_rounded'] = df_targets['target_rating_gain'].round(-2)
df_targets['target_rating_gain_squared'] = df_targets['target_rating_gain']**2
print(df_targets.shape)
df_targets.head()

In [ ]:
df_targets.groupby("rating_latest_rounded")['target_rating_gain'].describe().round()

In [ ]:
(ggplot(df_targets.sample(1000),aes(x='rating_latest',y='target_rating')) +
 geom_point() +
 scale_x_continuous(breaks=list(range(800,2500,200))) +
  scale_y_continuous(breaks=list(range(800,2500,200)))

        
       )

In [ ]:
df_temp = df_targets.copy()
df_temp = df_temp.merge(df_outcomes,on=['user_id','time_control'],how='outer',suffixes=['_latest','_future'])
df_temp.head()

In [ ]:
# Successes - filter to where future rating >= target rating, then take earliest date for each user/time control
df_successes = df_temp.query('rating>=target_rating').sort_values("date_future").drop_duplicates(['user_id','time_control','target_rating'])
print(df_successes.shape)
df_successes.sample(5)

In [ ]:
# Successes and failures 
df_bin = df_targets.merge(df_successes[['user_id','time_control','target_rating','date_future']],on=['user_id','time_control','target_rating'],how='left')
# Was the target rating achieved?
df_bin['y_bin'] = df_bin['date_future'].notna().astype(int)
# If so, when?
df_bin['y_cont'] = (df_bin['date_future']-max_training_date).dt.days
print(df_bin.shape)
df_bin.sample(10)

In [ ]:
df_cont = df_bin[df_bin['y_bin']==1].copy()

# EDA

In [ ]:
y_bin_by_rating = df_bin.groupby(["rating_latest_rounded","time_control"])['y_bin'].agg([np.mean,len])
(ggplot(y_bin_by_rating[y_bin_by_rating['len']>=25].reset_index(),
        aes(x='rating_latest_rounded',y='mean',color='time_control')) +
 geom_point() +
      scale_x_continuous(breaks=list(range(600,2600,200))) +
         ylim([0,1])
       )

In [ ]:
y_bin_by_gain = df_bin.groupby(["target_rating_gain_rounded","time_control"])['y_bin'].agg([np.mean,len])
(ggplot(y_bin_by_gain[y_bin_by_gain['len']>=25].reset_index(),
        aes(x='target_rating_gain_rounded',y='mean',color='time_control')) +
 geom_point() +
         ylim([0,1])
       )

In [ ]:
bin_by_quant_vars = df_bin.groupby(['target_rating_gain_rounded','rating_latest_rounded'])['y_bin'].mean().reset_index().round(2)
bin_by_quant_vars.pivot(index='target_rating_gain_rounded',columns='rating_latest_rounded',values='y_bin').iloc[:-2,6:-7]

In [ ]:
(ggplot(bin_by_quant_vars.query("rating_latest_rounded>=1200"),aes(x='target_rating_gain_rounded',y='y_bin',group='rating_latest_rounded',
       color='rating_latest_rounded')) +
geom_line())

As target rating gain increases, the effect of latest rating should go from zero to more negative.

In [ ]:
mean_days_by_quant_vars = df_cont.groupby(['target_rating_gain_rounded','rating_latest_rounded'])['y_cont'].mean().reset_index().round()
mean_days_by_quant_vars.pivot(index='target_rating_gain_rounded',columns='rating_latest_rounded',values='y_cont').iloc[:-1,6:-6]

In [ ]:
mean_days_by_quant_vars.head()

In [ ]:
y_cont_by_rating = df_cont.groupby(["rating_latest_rounded",'time_control'])['y_cont'].agg([np.mean,len]).reset_index()
y_cont_by_rating.head()

In [ ]:
(ggplot(y_cont_by_rating.query('len>=25'),aes(x='rating_latest_rounded',y='mean',color='time_control')) +
        geom_point() +
    ylim(0,500) +
     scale_x_continuous(breaks=list(range(600,2600,200)))
        
       )

In [ ]:
mean_outcome_by_group = df_cont.groupby(['target_rating_gain_rounded','time_control','rating_latest_rounded'])['y_cont'].agg([np.mean,len]).reset_index()
(ggplot(mean_outcome_by_group.query('len>=30'),aes(x='target_rating_gain_rounded',y='mean')) +
      geom_point())

In [ ]:
# Outlier checks
## High gains
df_cont['target_rating_gain'].describe(percentiles=[x/10 for x in range(10)])

In [ ]:
## Quick gains
df_cont[df_cont['target_rating_gain']>100]['y_cont'].describe(percentiles=[x/10 for x in range(10)])

# Modeling

In [ ]:
class_predictors = ['target_rating_gain','rating_latest','blitz','bullet','rapid','rating_peak_diff','rating_180_diff']
df_bin_predictors = sm.add_constant(df_bin[class_predictors])
#logit = sm.Logit(endog=df_bin['y_bin'],exog=df_bin_predictors).fit()
logit = smf.logit(formula="""
y_bin~target_rating_gain*rating_latest*bullet+target_rating_gain*rating_latest*blitz+
target_rating_gain*rating_latest*classical+

rating_peak_diff*target_rating_gain+rating_180_diff*bullet+rating_180_diff*blitz+

target_rating_gain_squared*rating_latest*bullet
""",data=df_bin).fit()
logit.summary()

In [ ]:
# Regression
regress_predictors = ['target_rating_gain','rating_latest','blitz','bullet','rapid','rating_peak_diff','rating_90_diff']
df_cont_predictors = sm.add_constant(df_cont[regress_predictors])
#ols = sm.OLS(endog=df_cont['y_cont'],exog=df_cont_predictors).fit()
ols = smf.ols(formula="""
y_cont~target_rating_gain_squared+
rating_latest_squared+
target_rating_gain*rating_latest*bullet+target_rating_gain*rating_latest*blitz+target_rating_gain*rating_latest*classical+
rating_peak_diff*bullet+rating_peak_diff*blitz+rating_peak_diff*classical+
rating_90_diff*blitz+rating_90_diff*classical

""",data=df_cont).fit()
ols.summary()

# Evaluation

## Classification Evaluation

In [ ]:
df_bin['prob'] = logit.predict(df_bin)
df_bin['prob'].describe()

In [ ]:
print(round(df_bin['prob'].mean(),3))
print(round(df_bin['y_bin'].mean(),3))

In [ ]:
round(((df_bin['prob']-df_bin['y_bin'])**2).mean(),3)

In [ ]:
for x in df_bin['time_control'].unique():
    data = df_bin.query("time_control==@x")
    print(x)
    print(round(((data['prob']-data['y_bin'])**2).mean(),3))

In [ ]:
df_bin['decile'] = pd.qcut(df_bin['prob'],q=10)
deciles = df_bin.groupby('decile')[['prob','y_bin']].mean().reset_index()
deciles['index'] = np.arange(len(deciles))
decile_probs = deciles[['prob','index']].rename(columns={"prob":"value"})
decile_probs['variable'] = 'prob'
decile_actuals = deciles[['y_bin','index']].rename(columns={"y_bin":"value"})
decile_actuals['variable'] = 'actual'
deciles = pd.concat([decile_probs,decile_actuals],axis=0)
(ggplot(deciles,aes(x='index',y='value',fill='variable')) +
 geom_bar(stat='identity',position='dodge')
)

In [ ]:
(ggplot(df_bin.query('y_bin==0'),aes(x='prob')) +
geom_histogram(bins=20))

In [ ]:
# Identify the ones with high-prob that are zeros (where we're overpredicting)
df_bin.query("prob>=.75&y_bin==0").groupby(["time_control","rating_latest_rounded_300","target_rating_gain_rounded"]).size().sort_values(ascending=False).head(10)

In [ ]:
(ggplot(df_bin.query('y_bin==1'),aes(x='prob')) +
geom_histogram(bins=20))

In [ ]:
(ggplot(df_bin,aes(x='prob')) +
geom_histogram(bins=20))

In [ ]:
df_bin['rating_latest_rounded_200'] = 200*np.ceil(df_bin['rating_latest_rounded']/200).astype(int)
df_bin['rating_latest_rounded_300'] = 300*np.ceil(df_bin['rating_latest_rounded']/300).astype(int)
xtabs_bin = df_bin.groupby(['time_control','target_rating_gain_rounded','rating_latest_rounded_200'])[['y_bin','prob']].agg([np.mean,len]).iloc[:,:-1].round(2)
xtabs_bin.columns = ['prop_actual','n','mean_prob']
xtabs_bin.reset_index(inplace=True)
xtabs_bin['diff'] = xtabs_bin['mean_prob'] - xtabs_bin['prop_actual']
xtabs_bin['abs_diff'] = xtabs_bin['diff'].abs()
xtabs_bin.query("n>=50").sort_values("abs_diff",ascending=False).head(10)

In [ ]:
df_bin.groupby(['rating_latest_rounded_200'])[['y_bin','prob']].agg([np.mean,len]).iloc[:,:-1].round(2)


In [ ]:
df_bin.groupby(['target_rating_gain_rounded'])[['y_bin','prob']].agg([np.mean,len]).iloc[:,:-1].round(2)


In [ ]:
xtabs_bin.query("time_control=='Blitz'").pivot(index='target_rating_gain_rounded',columns='rating_latest_rounded_200',values='prop_actual').iloc[:-1,2:-2]

In [ ]:
xtabs_bin.query("time_control=='Blitz'").pivot(index='target_rating_gain_rounded',columns='rating_latest_rounded_200',values='mean_prob').iloc[:-1,2:-2]

In [ ]:
xtabs_bin.query("time_control=='Blitz'").pivot(index='target_rating_gain_rounded',columns='rating_latest_rounded_200',values='diff').iloc[:-1,2:-2]

## Regression Evaluation

In [ ]:
df_cont['pred'] = ols.predict(df_cont)
#df_cont.loc[df_cont['pred']<0,'pred'] = 0
df_cont['pred'].describe()

In [ ]:
len(df_cont[df_cont['pred']<0])/len(df_cont)

In [ ]:
df_cont['error'] = df_cont['pred']-df_cont['y_cont']
df_cont['abs_error'] = df_cont['error'].abs()

In [ ]:
df_cont['error'].describe().round()

In [ ]:
df_cont['abs_error'].describe().round()

In [ ]:
error_summary = df_cont.groupby(['target_rating_gain_rounded','rating_latest_rounded','time_control'])[['pred','y_cont','abs_error','error']].agg([np.mean]).round().astype(int)
sizes = df_cont.groupby(['target_rating_gain_rounded','rating_latest_rounded','time_control']).size().reset_index()
sizes.rename(columns={0:"n"},inplace=True)
error_summary.columns = ['mean_pred','mean_actual','mean_abs_error','mean_error']
error_summary = error_summary.reset_index().merge(sizes,on=['target_rating_gain_rounded','rating_latest_rounded','time_control'])
error_summary = error_summary.query('n>30')
error_summary.sort_values("mean_error",ascending=True).head()

In [ ]:
df_cont.groupby("time_control")['abs_error'].mean().round()

In [ ]:
error_summary.query("time_control=='Bullet'").pivot(index='target_rating_gain_rounded',columns='rating_latest_rounded',values='mean_actual').iloc[:-1,2:-2]

In [ ]:
error_summary.query("time_control=='Bullet'").pivot(index='target_rating_gain_rounded',columns='rating_latest_rounded',values='mean_pred').iloc[:-1,2:-2]

In [ ]:
error_summary.query("time_control=='Bullet'").pivot(index='target_rating_gain_rounded',columns='rating_latest_rounded',values='mean_error').iloc[:-1,2:-2]

In [ ]:
error_summary.query("time_control=='Bullet'").pivot(index='target_rating_gain_rounded',columns='rating_latest_rounded',values='n').iloc[:-1,2:-2]

In [ ]:
(ggplot(error_summary.query("time_control=='Blitz'&target_rating_gain_rounded<=300")) +
geom_point(aes(x='rating_latest_rounded',y='mean_pred'),color='red')+
 geom_point(aes(x='rating_latest_rounded',y='mean_actual'),color='blue')
)

# Scoring

In [ ]:
def process_rating_history(response_json):
    rating_history = dict()
    for x in response_json:
            if x['name'] in ['Bullet','Blitz','Rapid','Classical']:
                tbl = pd.DataFrame(x['points'])
                tbl.columns = ['year','month','day','rating']
                tbl['month'] = tbl['month']+1
                tbl['date'] = pd.to_datetime(tbl.year*10000+tbl.month*100+tbl.day,format='%Y%m%d')
                rating_history[x['name']] = tbl
    return(rating_history)

def get_prob_success(predictor_values):
    return 1

def get_predicted_days(predictor_values):
    predicted_days = 0
    for i in range(len(ols.params)):
        var_name = ols.params.index[i]
        coef = ols.params.values[i]
        if ':' in var_name:
            var_names = var_name.split(":")
            value = 1
            for j in var_names:
                value *= predictor_values[j]
        else:
            value = predictor_values[var_name]
        predicted_days += coef*value
    return(round(predicted_days))

def score(username,target_rating_gain,target_time_control):
    url = f'https://lichess.org/api/user/{username}/rating-history'
    response = requests.get(url)
    response_json = response.json()
    if response.status_code != 200:
        return(f"API ERROR: {response.status_code}")
    else:
        rating_history = process_rating_history(response_json)
        target_rating_history = rating_history[target_time_control]
        target_rating_history['today'] = datetime.today()
        rating_latest = target_rating_history['rating'].values[-1]
        predictor_values = dict(Intercept=1,target_rating_gain=target_rating_gain,
        target_rating_gain_squared=target_rating_gain**2,
        rating_latest=rating_latest,
        rating_latest_squared = rating_latest**2,
        bullet = int(target_time_control == 'bullet'), blitz = int(target_time_control == 'blitz'),
        rapid = int(target_time_control == 'rapid'), classical = int(target_time_control == 'classical'),
        rating_peak_diff = rating_latest-target_rating_history['rating'].max(),
        # TO DO
        rating_90_diff = 200#rating_latest-target_rating_history[target_rating_history['date']<=target_rating_history['today']].values[-1]

                           )
        prob_success = get_prob_success(predictor_values)
        predicted_days = get_predicted_days(predictor_values)
        return(prob_success,predicted_days)
score(username = "",target_rating_gain = 50,
      target_time_control = "Bullet")


In [ ]:
#datetime.date(datetime.today()-timedelta(days=90)).strftime(format='%Y-%m-%d')

In [ ]:
#target_rating_history.dtypes

## Features
- Target time control (likely interacted with various other features)
- Target rating gain
- Current rating (likely nonlinear relationship)
- Rating growth in last 30 days / 90 days / 180 days
- Rating volatility measures
- Peak historical rating relative to current rating
- Rating in other time controls + puzzles
- Rating growth in other time controls + puzzles
- Difference between other time control ratings + target time control rating
- How long you've been on lichess
- How many games you've played (ever, and within last 30 days, and within the target time control - if you haven't played many it could mean more uncertainty). Consider that most discord bot users will have played more recent rated games in the target time control than the typical user in the training data. 
- Last time you played a rated game in the target time control (if it's a long time ago, it could mean more uncertainty)

## Outcomes
- Will you ever achieve a rating that's X rating points higher than your current rating in the next Y months (X is calculated from target rating submitted by user, Y = 24?)
- If so, when will you first reach the target rating? (point estimate + prediction interval of dates) - use number of days as outcome, then transform to date for the bot message
- Try to tweak model to avoid negative predictions, and manually override when needed. Same with predictions greater than 2 years out.


## Notes:
- Use cross-validation since sample size might be constrained
- Need to impute nulls
- Need to write code for scoring based on discord input (including lichess API querying)
- Need to figure out how to make prediction interval
- Add more comments + documentation to final version
